In [60]:
import pandas as pd
import numpy as np
import time 

from bs4 import BeautifulSoup

import requests

from bs4 import Comment

from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [61]:
url = 'https://boardgamegeek.com/boardgame/199792/everdell'

In [62]:
min_players_sel = '#mainbody > div.global-body-content-container.container-fluid > div > div.content.ng-isolate-scope > div:nth-child(2) > ng-include > div > ng-include > div > div.game-header > div.game-header-body > div.game-header-gameplay.hidden-game-header-collapsed.ng-scope > gameplay-module > div > div > ul > li:nth-child(1) > p > span > span:nth-child(1)'
max_players_sel = '#mainbody > div.global-body-content-container.container-fluid > div > div.content.ng-isolate-scope > div:nth-child(2) > ng-include > div > ng-include > div > div.game-header > div.game-header-body > div.game-header-gameplay.hidden-game-header-collapsed.ng-scope > gameplay-module > div > div > ul > li:nth-child(1) > p > span > span:nth-child(2)'
min_duration_sel = '#mainbody > div.global-body-content-container.container-fluid > div > div.content.ng-isolate-scope > div:nth-child(2) > ng-include > div > ng-include > div > div.game-header > div.game-header-body > div.game-header-gameplay.hidden-game-header-collapsed.ng-scope > gameplay-module > div > div > ul > li:nth-child(2) > p > span > span > span:nth-child(1)'
max_duration_sel = '#mainbody > div.global-body-content-container.container-fluid > div > div.content.ng-isolate-scope > div:nth-child(2) > ng-include > div > ng-include > div > div.game-header > div.game-header-body > div.game-header-gameplay.hidden-game-header-collapsed.ng-scope > gameplay-module > div > div > ul > li:nth-child(2) > p > span > span > span:nth-child(2)'
weight_sel_medium = '#mainbody > div.global-body-content-container.container-fluid > div > div.content.ng-isolate-scope > div:nth-child(2) > ng-include > div > ng-include > div > div.game-header > div.game-header-body > div.game-header-gameplay.hidden-game-header-collapsed.ng-scope > gameplay-module > div > div > ul > li:nth-child(4) > p > span.ng-isolate-scope > span.ng-binding.gameplay-weight-medium'
weight_sel_light = '#mainbody > div.global-body-content-container.container-fluid > div > div.content.ng-isolate-scope > div:nth-child(2) > ng-include > div > ng-include > div > div.game-header > div.game-header-body > div.game-header-gameplay.hidden-game-header-collapsed.ng-scope > gameplay-module > div > div > ul > li:nth-child(4) > p > span.ng-isolate-scope > span.ng-binding.gameplay-weight-light'
weight_sel_heavy = '#mainbody > div.global-body-content-container.container-fluid > div > div.content.ng-isolate-scope > div:nth-child(2) > ng-include > div > ng-include > div > div.game-header > div.game-header-body > div.game-header-gameplay.hidden-game-header-collapsed.ng-scope > gameplay-module > div > div > ul > li:nth-child(4) > p > span.ng-isolate-scope > span.ng-binding.gameplay-weight-heavy'


In [63]:
weight_sels = [weight_sel_light, weight_sel_medium, weight_sel_heavy]

In [64]:
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--disable-extensions")


driver = webdriver.Chrome(options=chrome_options)


In [65]:
db = pd.read_csv('database.csv', index_col=0)

In [66]:
db.head()

,Name,Rank,Link,Geek Rating,Average ratings,Voters
0,Brass: Birmingham,1,https://boardgamegeek.com/boardgame/224517/bra...,8.413,8.59,48020.0
1,Pandemic Legacy: Season 1,2,https://boardgamegeek.com/boardgame/161936/pan...,8.376,8.52,54158.0
2,Gloomhaven,3,https://boardgamegeek.com/boardgame/174430/glo...,8.344,8.58,63133.0
3,Ark Nova,4,https://boardgamegeek.com/boardgame/342942/ark...,8.340,8.54,46419.0
4,Twilight Imperium: Fourth Edition,5,https://boardgamegeek.com/boardgame/233078/twi...,8.237,8.59,24561.0


In [67]:
min_players = []
max_players = []
min_duration = []
max_duration = []
weight = []

In [68]:
def find_last_number(s):
    result = ''
    for i in range(len(s)-1,0,-1):
        if s[i]=='>':
            return result
        else:
            result=s[i]+result

In [69]:
i = 0
for link in db['Link']:
        #time.sleep(1)
        driver.get(link)
        try:
            element = WebDriverWait(driver, 1).until(EC.visibility_of_element_located((By.CSS_SELECTOR, min_players_sel)))
            #print(element.get_attribute("innerHTML"))
            min_players_num = int(element.get_attribute("innerHTML"))
            min_players.append(min_players_num)
        except:
            min_players.append('na')
        try:
            element = driver.find_element(By.CSS_SELECTOR, max_players_sel)
            max_players_num = int(find_last_number(element.get_attribute("innerHTML")))
            max_players.append(max_players_num)
        except:
            max_players.append('na')
        try:
            element = driver.find_element(By.CSS_SELECTOR, min_duration_sel)
            min_duration_num = int(element.get_attribute("innerHTML"))
            min_duration.append(min_duration_num)
        except:
            min_duration.append('na')
        try:
            element = driver.find_element(By.CSS_SELECTOR, max_duration_sel)
            max_duration_num = int(find_last_number(element.get_attribute("innerHTML")))
            max_duration.append(max_duration_num)
        except:
            max_duration.append('na')
        try:
            for sel in weight_sels:
                elements = driver.find_elements(By.CSS_SELECTOR, sel)
                if elements:
                    element = elements[0]
                    text_html = element.get_attribute("innerHTML")
                    text = element.get_attribute('innerText') 
                    weight.append(float(text.strip()))
                    print(link, text)

        except:
            print('aa')
            weight.append('na')  
        i+=1
  
driver.quit()


https://boardgamegeek.com/boardgame/224517/brass-birmingham 3.87 
https://boardgamegeek.com/boardgame/161936/pandemic-legacy-season-1 2.83 
https://boardgamegeek.com/boardgame/174430/gloomhaven 3.91 
https://boardgamegeek.com/boardgame/342942/ark-nova 3.78 
https://boardgamegeek.com/boardgame/233078/twilight-imperium-fourth-edition 4.33 
https://boardgamegeek.com/boardgame/316554/dune-imperium 3.06 
https://boardgamegeek.com/boardgame/167791/terraforming-mars 3.27 
https://boardgamegeek.com/boardgame/115746/war-of-the-ring-second-edition 4.21 
https://boardgamegeek.com/boardgame/187645/star-wars-rebellion 3.74 
https://boardgamegeek.com/boardgame/291457/gloomhaven-jaws-of-the-lion 3.64 
https://boardgamegeek.com/boardgame/162886/spirit-island 4.07 
https://boardgamegeek.com/boardgame/220308/gaia-project 4.40 
https://boardgamegeek.com/boardgame/12333/twilight-struggle 3.61 
https://boardgamegeek.com/boardgame/182028/through-the-ages-a-new-story-of-civilization 4.44 
https://boardgamege

In [70]:
len(weight)


1000

In [71]:
db['min_layers'] = min_players
db['max_layers'] = max_players
db['min_duration'] = min_duration
db['max_duration'] = max_duration
db['weight'] = weight

In [72]:
db.to_csv('database_duration_and_weight.csv')